# Natural Language Inference With BERT

## For this homework, we will work on (NLI)[https://nlp.stanford.edu/projects/snli/].

The task is, give two sentences: a premise and a hypothesis, to classify the relation between them. We have three classes to describe this relationship.

1. Entailment: the hypothesis follows from the fact that the premise is true
2. Contradiction: the hypothesis contradicts the fact that the premise is true
3. Neutral: There is not relationship between premise and hypothesis

See below for examples

![snli_task](media/snli_task.jpeg)

### First let's load the Stanford NLI dataset from the huggingface datasets hub using the datasets package

### Explore the dataset!

In [1]:
# Imports for most of the notebook
import torch
from transformers import BertModel
from transformers import AutoTokenizer
from typing import Dict, List
from util import get_snli

In [2]:
from datasets import load_dataset
dataset = load_dataset("snli")
print("Split sizes (num_samples, num_labels):\n", dataset.shape)
print("\nExample:\n", dataset['train'][0])

Found cached dataset snli (C:/Users/Maddie/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/3 [00:00<?, ?it/s]

Split sizes (num_samples, num_labels):
 {'test': (10000, 3), 'train': (550152, 3), 'validation': (10000, 3)}

Example:
 {'premise': 'A person on a horse jumps over a broken down airplane.', 'hypothesis': 'A person is training his horse for a competition.', 'label': 1}


Each example is a dictionary with the keys: (premise, hypothesis, label). 

#### Data Fields
 - premise: a string used to determine the truthfulness of the hypothesis
 - hypothesis: a string that may be true, false, or whose truth conditions may not be knowable when compared to the premise
 - label: an integer whose value may be either 0, indicating that the hypothesis entails the premise, 1, indicating that the premise and hypothesis neither entail nor contradict each other, or 2, indicating that the hypothesis contradicts the premise.

In [3]:
from util import get_snli

train_dataset, validation_dataset, test_dataset = get_snli()

Found cached dataset snli (C:/Users/Maddie/.cache/huggingface/datasets/snli/plain_text/1.0.0/1f60b67533b65ae0275561ff7828aad5ee4282d0e6f844fd148d05d3c6ea251b)


  0%|          | 0/3 [00:00<?, ?it/s]

Reading Datapoints: 100%|█████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 17580.55it/s]


In [4]:
## sub set stats
from collections import Counter

# num sample stats
print(len(train_dataset), len(validation_dataset), len(test_dataset))

# label distribution
print(Counter([t['label'] for t in train_dataset]))
print(Counter([t['label'] for t in validation_dataset]))
print(Counter([t['label'] for t in test_dataset]))

# We have a perfectly balanced dataset

9999 999 999
Counter({0: 3333, 1: 3333, 2: 3333})
Counter({0: 333, 1: 333, 2: 333})
Counter({0: 333, 1: 333, 2: 333})


### We want a function to load samples from the huggingface dataset so that they can be batched and encoded for our model.

### Now let's reimplement our tokenizer using the huggingface tokenizer.

### Notice that our __call__ method (the one called when we call an instance of our class) takes both a premise batch and a hypothesis batch.
### The HuggingFace BERT tokenizer knows to join these with the special sentence seperator token between them. We let HuggingFace do most of the work here for making batches of tokenized and encoded sentences. 

In [5]:
# Nothing to do for this class!

class BatchTokenizer:
    """Tokenizes and pads a batch of input sentences."""

    def __init__(self):
        """Initializes the tokenizer

        Args:
            pad_symbol (Optional[str], optional): The symbol for a pad. Defaults to "<P>".
        """
        self.hf_tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-small")
    
    def get_sep_token(self,):
        return self.hf_tokenizer.sep_token
    
    def __call__(self, prem_batch: List[str], hyp_batch: List[str]) -> List[List[str]]:
        """Uses the huggingface tokenizer to tokenize and pad a batch.

        We return a dictionary of tensors per the huggingface model specification.

        Args:
            batch (List[str]): A List of sentence strings

        Returns:
            Dict: The dictionary of token specifications provided by HuggingFace
        """
        # The HF tokenizer will PAD for us, and additionally combine 
        # The two sentences deimited by the [SEP] token.
        enc = self.hf_tokenizer(
            prem_batch,
            hyp_batch,
            padding=True,
            return_token_type_ids=False,
            return_tensors='pt'
        )

        return enc
    

# HERE IS AN EXAMPLE OF HOW TO USE THE BATCH TOKENIZER
tokenizer = BatchTokenizer()
x = tokenizer(*[["this is the premise.", "This is also a premise"], ["this is the hypothesis", "This is a second hypothesis"]])
print(x)
tokenizer.hf_tokenizer.batch_decode(x["input_ids"])



{'input_ids': tensor([[  101,  2023,  2003,  1996, 18458,  1012,   102,  2023,  2003,  1996,
         10744,   102,     0],
        [  101,  2023,  2003,  2036,  1037, 18458,   102,  2023,  2003,  1037,
          2117, 10744,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


['[CLS] this is the premise. [SEP] this is the hypothesis [SEP] [PAD]',
 '[CLS] this is also a premise [SEP] this is a second hypothesis [SEP]']

### We can batch the train, validation, and test data, and then run it through the tokenizer

In [6]:
def generate_pairwise_input(dataset: List[Dict]) -> (List[str], List[str], List[int]):
    """
    TODO: group all premises and corresponding hypotheses and labels of the datapoints
    a datapoint as seen earlier is a dict of premis, hypothesis and label
    """
    
    tokenizer = BatchTokenizer()
    sep = tokenizer.get_sep_token()
    prems = []
    hyps = []
    labs = []
    for item in dataset:
        prem = item['premise']
        prems.append(prem)
        hyp = item['hypothesis']
        hyps.append(hyp)
        lab = item['label']
        labs.append(lab)
    
    return (prems, hyps, labs)

In [7]:
train_premises, train_hypotheses, train_labels = generate_pairwise_input(train_dataset)
validation_premises, validation_hypotheses, validation_labels = generate_pairwise_input(validation_dataset)
test_premises, test_hypotheses, test_labels = generate_pairwise_input(test_dataset)

In [8]:
def chunk(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def chunk_multi(lst1, lst2, n):
    for i in range(0, len(lst1), n):
        yield lst1[i: i + n], lst2[i: i + n]
        
# Notice that since we use huggingface, we tokenize and
# encode in all at once!

### Let's batch the labels, ensuring we get them in the same order as the inputs

In [9]:
def encode_labels(labels: List[int]) -> torch.FloatTensor:
    """Turns the batch of labels into a tensor

    Args:
        labels (List[int]): List of all labels in the batch

    Returns:
        torch.FloatTensor: Tensor of all labels in the batch
    """
    return torch.LongTensor([int(l) for l in labels])


### Now we implement the model. Notice the TODO and the optional TODO (read why you may want to do this one.)

In [10]:
class NLIClassifier(torch.nn.Module):
    def __init__(self, output_size: int, hidden_size: int):
        super().__init__()
        self.output_size = output_size
        self.hidden_size = hidden_size
        # Initialize BERT, which we use instead of a single embedding layer.
        self.bert = BertModel.from_pretrained("prajjwal1/bert-small")
        # TODO [OPTIONAL]: Updating all BERT parameters can be slow and memory intensive. 
        # Freeze them if training is too slow. Notice that the learning
        # rate should probably be smaller in this case.
        # Uncommenting out the below 2 lines means only our classification layer will be updated.
        # for param in self.bert.parameters():
        #     param.requires_grad = False
        self.bert_hidden_dimension = self.bert.config.hidden_size
        # TODO: Add an extra hidden layer in the classifier, projecting
        #      from the BERT hidden dimension to hidden size.
        self.hidden_layer = torch.nn.Linear(self.bert_hidden_dimension, self.hidden_size)
        # TODO: Add a relu nonlinearity to be used in the forward method
        #      https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html
        self.relu = torch.nn.ReLU()
        self.classifier = torch.nn.Linear(self.hidden_size, self.output_size)
        self.log_softmax = torch.nn.LogSoftmax(dim=2)

    def encode_text(
        self,
        symbols: Dict
    ) -> torch.Tensor:
        """Encode the (batch of) sequence(s) of token symbols with an LSTM.
            Then, get the last (non-padded) hidden state for each symbol and return that.

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: The final hiddens tate of the LSTM, which represents an encoding of
                the entire sentence
        """
        # First we get the contextualized embedding for each input symbol
        # We no longer need an LSTM, since BERT encodes context and 
        # gives us a single vector describing the sequence in the form of the [CLS] token.
        encoded_sequence = self.bert(**symbols)
        # TODO: Get the [CLS] token using the `pooler_output` from 
        #      The BertModel output. See here: https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
        #      and check the returns for the forward method.
        self.cls = encoded_sequence.pooler_output
        self.output = self.cls[:, None, :]
        
        # We want to return a tensor of the form batch_size x 1 x bert_hidden_dimension

        return self.output

    def forward(
        self,
        symbols: Dict,
    ) -> torch.Tensor:
        """_summary_

        Args:
            symbols (Dict): The Dict of token specifications provided by the HuggingFace tokenizer

        Returns:
            torch.Tensor: _description_
        """
        encoded_sents = self.encode_text(symbols)
        output = self.hidden_layer(encoded_sents)
        output = self.relu(output)
        output = self.classifier(output)
        return self.log_softmax(output)

In [11]:
# For making predictions at test time
def predict(model: torch.nn.Module, sents: torch.Tensor) -> List:
    logits = model(sents)
    return list(torch.argmax(logits, axis=2).squeeze().numpy())

### Evaluation metrics: Macro F1

In [12]:
import numpy as np
from numpy import logical_and, sum as t_sum


def precision(predicted_labels, true_labels, which_label=1):
    """
    Precision is True Positives / All Positives Predictions
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(pred_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def recall(predicted_labels, true_labels, which_label=1):
    """
    Recall is True Positives / All Positive Labels
    """
    pred_which = np.array([pred == which_label for pred in predicted_labels])
    true_which = np.array([lab == which_label for lab in true_labels])
    denominator = t_sum(true_which)
    if denominator:
        return t_sum(logical_and(pred_which, true_which))/denominator
    else:
        return 0.


def f1_score(
    predicted_labels: List[int],
    true_labels: List[int],
    which_label: int
):
    """
    F1 score is the harmonic mean of precision and recall
    """
    P = precision(predicted_labels, true_labels, which_label=which_label)
    R = recall(predicted_labels, true_labels, which_label=which_label)
    if P and R:
        return 2*P*R/(P+R)
    else:
        return 0.


def macro_f1(
    predicted_labels: List[int],
    true_labels: List[int],
    possible_labels: List[int]
):
    scores = [f1_score(predicted_labels, true_labels, l) for l in possible_labels]
    # Macro, so we take the uniform avg.
    return sum(scores) / len(scores)

### Training loop.

In [13]:
import random
import tqdm

def training_loop(
    num_epochs,
    train_features,
    train_labels,
    dev_sents,
    dev_labels,
    optimizer,
    model,
    possible_labels,
):
    print("Training...")
    loss_func = torch.nn.NLLLoss()
    batches = list(zip(train_features, train_labels))
    random.shuffle(batches)
    for i in range(num_epochs):
        losses = []
        print("Working on epoch", i)
        for features, labels in tqdm.tqdm(batches):
            # Empty the dynamic computation graph
            optimizer.zero_grad()
            preds = model(features).squeeze(1)
            loss = loss_func(preds, labels)
            # Backpropogate the loss through our model
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        
        print(f"epoch {i}, loss: {sum(losses)/len(losses)}")
        # Estimate the f1 score for the development set
        print("Evaluating dev...")
        all_preds = []
        all_labels = []
        for sents, labels in tqdm.tqdm(zip(dev_sents, dev_labels), total=len(dev_sents)):
            pred = predict(model, sents)
            all_preds.extend(pred)
            all_labels.extend(list(labels.numpy()))

        dev_f1 = macro_f1(all_preds, all_labels, possible_labels)
        print(f"Dev F1 {dev_f1}")
        
    # Return the trained model
    return model

In [14]:
# You can increase epochs if need be
epochs = 10
# TODO: Find a good learning rate
LR = 0.00001
hidden_size = 64

tokenizer = BatchTokenizer()
batch_size = 32
train_input_batches = [b for b in chunk_multi(train_premises, train_hypotheses, batch_size)]
# Tokenize + encode
train_input_batches = [tokenizer(*batch) for batch in train_input_batches]
train_label_batches = [b for b in chunk(train_labels, batch_size)]
train_label_batches = [encode_labels(batch) for batch in train_label_batches]

possible_labels = set(train_labels)
num_possible_labels = len(set(train_labels))
model = NLIClassifier(num_possible_labels, hidden_size)
optimizer = torch.optim.AdamW(model.parameters(), LR)

validation_input_batches = [b for b in chunk_multi(validation_premises, validation_hypotheses, batch_size)]
# Tokenize + encode
validation_input_sents = [tokenizer(*batch) for batch in validation_input_batches]
validation_batch_labels = [b for b in chunk(validation_labels, batch_size)]
validation_encoded_labels = [encode_labels(batch) for batch in validation_batch_labels]

output_model = training_loop(
    epochs,
    train_input_batches,
    train_label_batches,
    validation_input_sents,
    validation_encoded_labels,
    optimizer,
    model,
    possible_labels
)

Some weights of the model checkpoint at prajjwal1/bert-small were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Training...
Working on epoch 0


100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [17:17<00:00,  3.31s/it]


epoch 0, loss: 1.0936015006452322
Evaluating dev...


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:41<00:00,  1.30s/it]


...

Dev F1 0.7194820219589442
Working on epoch 9


100%|████████████████████████████████████████████████████████████████████████████████| 313/313 [20:06<00:00,  3.85s/it]


epoch 9, loss: 0.1801947175849455
Evaluating dev...


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:45<00:00,  1.41s/it]

Dev F1 0.7174082486592441


In [15]:
# TODO: Get a final macro F1 on the test set.
# You should be able to mimic what we did with the validaiton set.

def eval_test_set(test_sents, test_labels, possible_labels):
    print("Evaluating test set...")
    all_preds = []
    all_labels = []
    for sents, labels in tqdm.tqdm(zip(test_sents, test_labels), total=len(test_sents)):
        pred = predict(output_model, sents)
        all_preds.extend(pred)
        all_labels.extend(list(labels.numpy()))

    dev_f1 = macro_f1(all_preds, all_labels, possible_labels)
    print(f"Dev F1 {dev_f1}")


In [16]:
test_premises, test_hypotheses, test_labels = generate_pairwise_input(test_dataset)
test_input_batches = [b for b in chunk_multi(test_premises, test_hypotheses, batch_size)]
# Tokenize + encode
test_input_sents = [tokenizer(*batch) for batch in test_input_batches]
test_batch_labels = [b for b in chunk(test_labels, batch_size)]
test_encoded_labels = [encode_labels(batch) for batch in test_batch_labels]

eval_test_set(test_input_sents, test_encoded_labels, possible_labels)

Evaluating test set...


100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:25<00:00,  1.24it/s]

Dev F1 0.693278348207397


# Written Assignment

### 1. Describe the task and what capability is required to solve it.

The purpose of this task is to create a model that can evaluate the validity of a hypothesis based upon a given premise. For example, given the premise "The cat is being chased by a dog," the model can determine whether the hypothesis "The dog is taking a nap" is true. Each premise-hypothesis pairing is put into one of the three following classes: entailment if the hypothesis must be true given the premise, contradiction if the hypothesis cannot be true given the premise, or neutral if the premise has no bearing on the truth of the hypothesis. However, this task is somewhat subjective and not all human evaluators agree on the correct characterization, as indicated by the human judgements given at the beginning of this assignment. The human judgements differed from the majority opinion 12% of the time, so the ideal F1 score would be 0.88.
To solve this task, the model must be able to determine both the meanings of the words and how the meanings interact with one another. For example, it must be able to understand that with the premise given above, the hypothesis "The dog is chasing the cat" is an entailment while "The cat is chasing the dog" is a contradiction (in addition to the meaning-based contradiction above). Thus the model must have some grasp on both semantics and grammatical relations. 

### 2. How does the method of encoding sequences of words in our model differ here, compared to the word embeddings in HW 4. What is different? Why benefit does this method have?

In this model, we have added an extra hidden layer which allows the model to learn more complex relations between tokens. While the word-to-vec embeddings encodes each word's meaning and the first hidden layer encodes something similar to grammatical relations, the second hidden layer allows the model to give more meaning to the grammatical relations. The addition of the second hidden layer helps the model distinguish between sentences with the same words and the same structure but with different orders such as "the cat chased the dog" and "the dog chased the cat."

### 3. Discuss your results. Did you do any hyperparameter tuning? Did the model solve the task?

Balancing time efficiency and F1 score was the most critical part of this task. Though freezing the parameters did slightly speed up the computation, it only sped it up to 14 minutes per epoch instead of 15 minutes per epoch and my F1 scores were abysmal (in the 0.50s). The most time-efficient and successful solution I found used the following parameters: a learning rate of 0.0001, a hidden layer of 64, and a batch size of 32. After 10 epochs (which took my very sad computer an 160 minutes to do), I was able to obtain macro F1 scores of 0.717 on the validation set (with a maximum of 0.736 on the sixth epoch) and 0.693 on the test set. The F1 scores had begun to flatten out by the 5th epoch, so I estimate that if I had had the time to run more epochs, it would likely stay around that value. 
Based upon the human judgements provided at the beginning in which judgments differed from the majority opinion 12% of the time, the model I have developed is not quite at the level of human evaluators. Adding another hidden layer could help the model have access to more nuance to get it closer to humanlike performance.